In [1]:
import matplotlib  

In [2]:
import numpy 
import pandas

In [3]:
import abc


class DecayFunction(metaclass=abc.ABCMeta):
    """
    Base class for decay functions, implemented with a strategy design pattern
    """
    def __init__(self, init_value):
        self._value = init_value

    @property
    def value(self):
        return self._value
  
    @abc.abstractmethod
    def decay(self):
        pass



class LinearDecay(DecayFunction):
    """
    Decaying with a constant rate
    """
    def __init__(self, start_value, rate):
        self.rate = rate
        super().__init__(start_value)

    def decay(self):
        self._value -= self.rate


class ExponentialDecay(DecayFunction):
    """
    Decaying exponential.
    """
    def __init__(self, start_value, rate):
        self.rate = rate
        super().__init__(start_value)

    def decay(self):
        self._value *= self.rate 

In [9]:
import math 
# def manhattan_distance_2d(x, y):
#     return abs(x[0] - y[0]) + abs(x[1] - y[1])


def euclidean_distance_2d(x, y):
    return math.sqrt(pow(y[0]-x[0],2) + pow(y[1]-x[1],2))
 

def euclidean_distance_1d(x, y):
    return abs(x-y)


def euclidean_distance_1d_circular(n, i, j):
    """
    Calculates 1d distance of integers i and j in a circle of n elements.
    :param n: Circle length
    :param i: Start element
    :param j: Target element
    :return: Minimal distance
    """
    d = euclidean_distance_1d(i, j)
    return min(d, n-d)

In [10]:
import math


def gaussian(d, sigma):
    p = sigma**2
    if p == 0:
        return 0
    return math.exp(-d**2/(2*p))


def bubble(d, sigma):
    if d <= sigma:
        return 1  
    return 0 

In [11]:
import matplotlib.pyplot as plt
import os 
# 
DEFAULTS_EXP = {'uruguay': ('uruguay', 8, 10000, 1000, 500, gaussian,
                        ExponentialDecay(0.7, 0.9999),
                        ExponentialDecay(734*8/10, 0.999)),
            'western_sahara': ('western_sahara', 8, 500, 50, 50, gaussian,
                        ExponentialDecay(0.7, 0.999),
                        ExponentialDecay(29*8/10, 0.995)),
            'qatar': ('qatar', 8, 5000, 500, 500, gaussian,
                        ExponentialDecay(0.9, 0.9999),
                        ExponentialDecay(194*8/10, 0.997)),
            'djibouti': ('djibouti', 8, 1000, 50, 50, gaussian, 
                        ExponentialDecay(0.7, 0.999),
                        ExponentialDecay(89*8/10, 0.995))}

plt.figure()


def plot_map(cities, neurons, iteration):
    """
    Generates the required map of cities and neurons at a given moment and
    stores the result in a png image. The map contains all the cities
    represented as red dots and all the neurons as green, crossed by a line
    dots. The method plots the iteration in which the snapshot was taken.
    :param cities: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param neurons: the cities to be plotted, passed as a list of (x, y)
    coordinates
    :param iteration: the iterations when the snapshot is taken
    :return: returns nothing
    """
    plt.scatter(*zip(*cities), color='red', s=3)
    plt.scatter(*zip(*neurons), color='green', s=2)

    plt.plot(*zip(*(neurons+[neurons[0]])), color='darkgreen')

    # Invert x axis to match representation at
    # http://www.math.uwaterloo.ca/tsp/world/countries.html
    plt.gca().invert_xaxis()
    plt.gca().set_aspect('equal', adjustable='datalim')

    plt.title('Iteration #{:06d}'.format(iteration))
    plt.axis('off')
    plt.savefig('results/{}.png'.format(iteration))
    plt.clf()


def read_data(filename):
    """
    Reads and parses data from a txt file with a map data. The format that the
    function expects is the one followed in the uwaterloo TSP web archive
    (math.uwaterloo.ca/tsp/world/countries.html), ignoring the first
    description lines that have to be manually removed. The method searches for
    the filename in the assets folder.
    :param filename: the path to the file to be parsed
    :return: the cities as a list of (x, y) coordinates
    """
    cities = []

    path = 'assets/{}.txt'.format(filename)
    with open(path, 'r') as f:
        for line in f:
            city = list(map(float, line.split()[1:]))
            cities.append((city[1], city[0]))

    return cities


def get_input():
    """
    Gets the input from the user line or launches the default values
    :return data_set: list of cities as (x,y) coordinates
    :return n_neurons: number of neurons per city in the data_set
    :return iterations: number of iterations to be executed
    :return learning_rate: learning rate to be used
    :return radius: radius of neurons to be used
    """
    data_sets = {'w': 'western_sahara', 'q': 'qatar', 'u': 'uruguay',
                 'd': 'djibouti'}

    set_id = input('Data set [w/q/u/d]: ') or 'w'
    data_set = data_sets[set_id]

    if not os.path.isfile('assets/{}.txt'.format(data_set)):
        exit("Did not find this data set file!")

    use_defaults = input('Do you want to use default parameters? (y/n) ') == 'y'

    if use_defaults:
        decay = input('What kind of decay? [s/l/e]') or 'e'
        if decay == 'e':
            return DEFAULTS_EXP[data_set] 
        if decay == 'l':
            return DEFAULTS_EXP[data_set]
        if decay == 's':
            return DEFAULTS_EXP[data_set] 
    # Comprehensive input
    n_neurons = int(input('How many neurons per city? (8)') or 8)
    iterations = int(input('How many iterations (500)?') or 500)
    learning_rate = get_input_decay('learning rate')
    radius = get_input_decay('radius')
    k = int(input('After how many iterations print current TSP distance? (1000)') or 1000)
    plot_k = int(input('After how many iterations generate plot? (200)') or 200)

    neighborhood = input('Choose neighborhood function: [g/b]') or 'b'
    if neighborhood == 'b':
        neighborhood = bubble
    elif neighborhood == 'g':
        neighborhood = gaussian
    else:
        exit('Not a valid neighborhood!')

    return data_set, n_neurons, iterations, k, plot_k, neighborhood, learning_rate, radius

def get_input_decay(name):
    """
    Generates the appropriate decay function for a given variable. The decays
    that can be generated are static (no decay over time), linear and
    exponential.
    :param name: name of the variable that will be generated
    :return: decay function for the variable with appropriate parameters
    """
    decay = input('What kind of decay for {}? [s/l/e]'.format(name)) or 'e'
    if decay == 's':
        value = float(input('Static value: '))
        return StaticDecay(value)
    if decay == 'l':
        value = float(input('Start value: '))
        rate = float(input('Rate:'))
        return LinearDecay(value, rate)
    if decay == 'e':
        value = float(input('Start value: '))
        rate = float(input('Rate:'))
        return ExponentialDecay(value, rate)
    exit('Not a valid option!')  

<Figure size 432x288 with 0 Axes>

In [12]:

# from helper import read_data, plot_map, get_input
# from distances import euclidean_distance_2d, euclidean_distance_1d_circular

import random
from operator import itemgetter
from functools import partial


def main():
    data_set, n_neurons, iterations, k, plot_k, neighborhood, learning_rate\
        , radius = get_input()
    cities = read_data(data_set)
    scaling, cities = normalize(cities)
    neuron_count = len(cities) * n_neurons
    neurons = init_neurons(neuron_count)

    # TODO maybe distance param?
    som(neurons, cities, iterations, k, plot_k, neighborhood, learning_rate, radius,
        scaling) 


def init_neurons(count):
    """
    Initialize the weights of the neurons
    :param count: number of neurons to initialize
    :return: list of neurons as weight vectors (x,y)
    """
    return [[random.uniform(0.0, 1.0),
             random.uniform(0.0, 1.0)] for i in range(count)]


def som(neurons, cities, iterations, k, plot_k, neighborhood, learning_rate,
        radius, scaling):
    """
    Main SOM loop to be executed.
    :param neurons: list of neurons as weight vectors (x,y)
    :param cities: list of cities as (x,y) coordinates
    :param iterations: number of iterations to perform
    :param k: lapse of iterations between printing execution data
    :param plot_k: lapse of iterations between saving the map images
    :param neighborhood: type of neighborhood to use in the execution
    :param learning_rate: learning rate to be used
    :param radius: radius of neurons to be used
    :param scaling: scale used when normalizing the data
    :return: returns nothing
    """
    for i in range(0, iterations+1):
        if i % k == 0:
            print('#', i, '\t\tTSP-distance: ',
                  calculate_tsp(cities, neurons)*scaling)
        if i % plot_k == 0:
            plot_map(cities, neurons, i)
        if i == iterations:
            break
        som_iteration(neurons, cities, neighborhood, learning_rate, radius)


def som_iteration(neurons, cities, neighborhood, learning_rate, radius):
    """
    Performs a single SOM iteration with a given set of parameters
    :param neurons: list of neurons as weight vectors (x,y)
    :param cities: list of cities as (x,y) coordinates
    :param neighborhood: type of neighborhood to use in the execution
    :param learning_rate: learning rate to be used
    :param radius: radius of neurons to be used
    :return: returns nothing
    """
    # Pick a random city
    city = cities[random.randint(0, len(cities)-1)]
    # Choose the winner neuron
    winner_index, winner = compute_winner(city, neurons, euclidean_distance_2d)
    distance = partial(euclidean_distance_1d_circular, len(neurons))

    # Update the weights of the neuron and its neighbourhood
    for neuron_index, neuron in enumerate(neurons):
        d = distance(neuron_index, winner_index)
        nf = neighborhood(d, radius.value)
        neuron[0] += learning_rate.value * nf * (city[0] - neuron[0])
        neuron[1] += learning_rate.value * nf * (city[1] - neuron[1])

    learning_rate.decay() 
    radius.decay()


def compute_winner(city, neurons, distance):
    """
    Computes the closest neuron to a given city with a given distance
    :param city: coordinates (x,y) of the city to be associated to the winner
    :param neurons: list of neurons as weight vectors (x,y)
    :param distance: type of distance to be used for the computations
    :return winner_index: index of the chosen neuron in the neuron list
    :return winner: closest neuron to city
    """
    return min([(i, distance(city, neuron)) for i,
                neuron in enumerate(neurons)], key=itemgetter(1))


def normalize(cities):
    """
    Normalize list of city coordinates
    :param cities: list of tuples, containing x and y coordinate
    :return: normalized list of city coordinates
    """
    max_x = max(cities,key=itemgetter(0))[0]
    max_y = max(cities,key=itemgetter(1))[1]
    m = max(max_x, max_y)

    return m, [(x/m, y/m) for (x, y) in cities]


def calculate_tsp(cities, neurons):
    """
    Computes the Travelling Salesman distance to travel all the cities using a
    list of neurons
    :param cities: list of tuples, containing x and y coordinate
    :param neurons: list of neurons as weight vectors (x,y)
    :return: the distance to travel through cities using neurons
    """
    city_neurons = {}
    for city_idx, city in enumerate(cities):
        # find nearest neuron
        idx, _ = compute_winner(city, neurons, euclidean_distance_2d)
        if idx not in city_neurons:
            city_neurons[idx] = [city]
        else: 
            city_neurons[idx].append(city)

    # order cities according to neuron order
    tsp_order = []
    for neuron_idx in range(len(neurons)):
        if neuron_idx in city_neurons:
            tsp_order += city_neurons[neuron_idx]

    # calculate tsp distance for tsp_order
    tsp_distance = euclidean_distance_2d(tsp_order[0], tsp_order[-1])
    for idx in range(len(tsp_order)-1):
        tsp_distance += euclidean_distance_2d(tsp_order[idx],
                                              tsp_order[idx + 1])

    return tsp_distance 

In [13]:
main()      

Data set [w/q/u/d]: 
Do you want to use default parameters? (y/n) y
What kind of decay? [s/l/e]e
# 0 		TSP-distance:  51790.21267043442
# 50 		TSP-distance:  31087.801727846578
# 100 		TSP-distance:  27823.485254979216
# 150 		TSP-distance:  28094.67343517503
# 200 		TSP-distance:  27601.173774493756
# 250 		TSP-distance:  27823.485254979216
# 300 		TSP-distance:  27739.383942485343
# 350 		TSP-distance:  27739.383942485343
# 400 		TSP-distance:  27739.383942485343
# 450 		TSP-distance:  27739.383942485343
# 500 		TSP-distance:  27739.383942485343


<Figure size 432x288 with 0 Axes>

In [20]:
# os.listdir(".") 

In [12]:
! mkdir "results"

In [ ]:
# I need to be able to understand; where some of these factors are coming from 